In [1]:
import pandas as pd
import os

from copy import copy
from openpyxl import load_workbook
from openpyxl.styles import Border,Side,Alignment,Font,PatternFill
from openpyxl.utils import get_column_letter

In [117]:
month,year = 2,2024

prof_tax = 150

contract_no ="22SNCJO-373"
pf,edli,esi,service_charges = 5293.5,211.74,1488.54,250.00

ld,penalty,taxes,other_recovery = 0,0,0,0

wages = pd.DataFrame({"CATEGORY":['SKILLED',"SEMI-SKILLED","UNSKILLED"],"Wage per day":[709,589,504]})

vendor_name = "M/S SPRYSOFT TECHNOLOGIES PRIVATE LIMITED, HYDERABAD."

month_st_dt = pd.Period(f"{year}-{month}").start_time.date().strftime("%d-%m-%Y")
month_end_dt = pd.Period(f"{year}-{month}").end_time.date().strftime("%d-%m-%Y")

month_year = pd.Period(f"{year}-{month}").start_time.date().strftime("%b-%Y")
month_line = f"Due Wage Month:{month_year} from {month_st_dt} to {month_end_dt}"

vendor_name_line = f"Name of Contractor : {vendor_name}"

attendance_path = os.path.join("attendance",f"attendance_format_{month}_{year}.xlsx")

template_path  = "attendance.xlsx"

In [170]:
def wage_calc_preprocessing(attendance_path,wages):
    pf_esi_data = pd.read_excel(attendance_path,sheet_name="PF_ESI",header=3).dropna(subset=["Name"])
    #wages["Days Present"] = pf_esi_data.loc[pf_esi_data.Name.str.contains("TOTAL"),"Days Present"].iloc[:-1].values
    mandays = []
    for category in wages.CATEGORY:
        values = pf_esi_data.loc[pf_esi_data.Name == f"{category} TOTAL","Days Present"].values
        if  values.size > 0 :
            mandays.append(values[0])
        else:
            mandays.append(0)
    wages["Days Present"] = mandays
    wages = wages.reindex(wages.index[::-1]).reset_index(drop=True)
    wages["SL NO"] = [int(val)+1  for val in wages.index]
    wages = pd.concat([wages.iloc[:,-1:],wages.iloc[:,:-1]],axis=1)
    wages["mandays_pf"] = wages["Days Present"]
    wages["total_gross_amount"] = wages["Wage per day"] * wages["Days Present"]
    return wages

In [171]:
wage_calc_df = wage_calc_preprocessing(attendance_path,wages)
wage_calc_df

,SL NO,CATEGORY,Wage per day,Days Present,mandays_pf,total_gross_amount
0,1,UNSKILLED,504,24.5,24.5,12348.0
1,2,SEMI-SKILLED,589,25.5,25.5,15019.5
2,3,SKILLED,709,26.0,26.0,18434.0


In [132]:
def generate_wage_calc_sheet(template_path,month,year,wage_calc_df,pf,esi,edli,service_charges,ld,penalty,taxes,other_recovery):
    
    month_year = pd.Period(f"{year}-{month}").start_time.date().strftime("%b-%Y")

    contract_line = f"Contract Reference: {contract_no}"
    vendor_line = f"Service Provider: {vendor_name}"
    month_line = f"Month: {month_year}"
    
    workbook = load_workbook(template_path)
    sheet = workbook['Wage_Calculation']  # Update with your sheet's name
    sheet.column_dimensions["B"].width = 35
    
    cell = sheet.cell(row=4,column=1)
    cell.value = contract_line

    cell = sheet.cell(row=5,column=1)
    cell.value = vendor_line

    cell = sheet.cell(row=6,column=1)
    cell.value = month_line
    
    start_row = 8
    start_column = 3

    for index, row in wage_calc_df.iloc[:,2:].iterrows():
        for col_num, value in enumerate(row, start=start_column): 

            cell = sheet.cell(row=start_row + index, column=col_num)
            cell.value = value
            
    start_row = 13
    start_column = 3

    for index, row in wage_calc_df[["Wage per day"]].iterrows():
        for col_num, value in enumerate(row, start=start_column): 

            cell = sheet.cell(row=start_row + index, column=col_num)
            cell.value = value
            
    cell = sheet.cell(row=16,column=6)
    cell.value = pf

    cell = sheet.cell(row=17,column=6)
    cell.value = esi

    cell = sheet.cell(row=18,column=6)
    cell.value = edli

    cell = sheet.cell(row=20,column=6)
    cell.value = service_charges
    
    sub_total = (wage_calc_df.sum()["total_gross_amount"] + pf + esi + edli + service_charges) 
    gst = sub_total * 0.18
    final_total = sub_total + gst
    
    cell = sheet.cell(row=21,column=6)
    cell.value = gst

    cell = sheet.cell(row=22,column=6)
    cell.value = final_total

    cell = sheet.cell(row=23,column=6)
    cell.value = final_total

    cell = sheet.cell(row=23,column=6)
    cell.value = ld

    cell = sheet.cell(row=24,column=6)
    cell.value = penalty

    cell = sheet.cell(row=25,column=6)
    cell.value = taxes

    cell = sheet.cell(row=26,column=6)
    cell.value = other_recovery
    
    total_recovery = ld + penalty + taxes + other_recovery
    cell = sheet.cell(row=27,column=6)
    cell.value = total_recovery
    
    net_payable = final_total - total_recovery

    cell = sheet.cell(row=28,column=6)
    cell.value = net_payable
    
    return sheet
    
    

    

    

In [134]:
sheet = generate_wage_calc_sheet(template_path,month,year,wage_calc_df,pf,esi,edli,service_charges,ld,penalty,taxes,other_recovery)

In [144]:
def create_wage_calc_sheet(attendance_path,sheet):
    attendance_workbook = load_workbook(attendance_path)

    try:
        if attendance_workbook["Wage_Calculation"]:
            attendance_workbook.remove(attendance_workbook["Wage_Calculation"])
            target_sheet = attendance_workbook.create_sheet(title = "Wage_Calculation")
    except:
        target_sheet = attendance_workbook.create_sheet(title = "Wage_Calculation")

    for row in sheet.iter_rows(values_only=True):
        target_sheet.append(row)

    for row in range(1,sheet.max_row+1):
        for col in range(1,sheet.max_column+1):
            source_cell = sheet.cell(row=row,column=col)
            target_cell = target_sheet.cell(row=row,column=col)
            target_cell.number_format = source_cell.number_format
            target_cell.font = copy(source_cell.font)
            target_cell.alignment = copy(source_cell.alignment)
            target_cell.border = copy(source_cell.border)
            target_cell.fill = copy(source_cell.fill)

    for col_idx,column in enumerate(sheet.columns,start=1):
        target_sheet.column_dimensions[get_column_letter(col_idx)].width = sheet.column_dimensions[get_column_letter(col_idx)].width
    
    for row in sheet.iter_rows():
        for cell in row:
            target_sheet.row_dimensions[cell.row].height = sheet.row_dimensions[cell.row].height
            
    for merged_cell_range in sheet.merged_cells.ranges:
        target_sheet.merge_cells(merged_cell_range.coord)
        for merged_cell in merged_cell_range.cells:
            target_cell = target_sheet.cell(row=merged_cell[0],column=merged_cell[1])
#             if merged_cell[0]  in [2,3]:
#                 target_cell.alignment = Alignment(horizontal="left",vertical="center")
#             elif merged_cell[0]  in [4,5]:
#                 target_cell.alignment = Alignment(horizontal="center",vertical="center",wrap_text=True)               
#             else:
            target_cell.alignment = Alignment(horizontal="left",vertical="center")

    attendance_workbook.save(attendance_path)
    return attendance_path

In [145]:
create_wage_calc_sheet(attendance_path,sheet)

'attendance\\attendance_format_2_2024.xlsx'

In [129]:
workbook.save("wage_calc.xlsx")

In [108]:
wage_calc_df.sum()["total_gross_amount"]

45801.5